In [1]:
!pip install -q torch diffusers transformers accelerate gTTS pydub opencv-python-headless numpy
!apt-get -y install ffmpeg

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 5.3 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.


In [2]:
import os
import re
from diffusers import DiffusionPipeline
import torch
from gtts import gTTS
from pydub import AudioSegment
import cv2
from IPython.display import HTML
from base64 import b64encode
import subprocess # Added subprocess import

/usr/local/lib/python3.12/dist-packages/pydub/utils.py:300: SyntaxWarning: invalid escape sequence '\('
  m = re.match('([su]([0-9]{1,2})p?) \(([0-9]{1,2}) bit\)$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:301: SyntaxWarning: invalid escape sequence '\('
  m2 = re.match('([su]([0-9]{1,2})p?)( \(default\))?$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:310: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(flt)p?( \(default\))?$', token):
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:314: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(dbl)p?( \(default\))?$', token):


In [3]:
os.makedirs("/content/images", exist_ok=True)
os.makedirs("/content/audio", exist_ok=True)


In [4]:
# Step 3: Input text

news_text = """
President Donald Trump announced the U.S. has seized an oil tanker off the coast of Venezuela.
Venezuela condemned the action, calling it an act of aggression.
International tensions are rising worldwide.
"""

In [5]:
# Step 4: Split text into sentences (regex-based)

def split_sentences(text):
    sentences = re.split(r'(?<=[.!?])\s+', text.strip())
    return [s for s in sentences if s]

sentences = split_sentences(news_text)
print("Sentences:", sentences)

Sentences: ['President Donald Trump announced the U.S.', 'has seized an oil tanker off the coast of Venezuela.', 'Venezuela condemned the action, calling it an act of aggression.', 'International tensions are rising worldwide.']


In [6]:
# Step 5: Load Diffusers model
device = "cuda" if torch.cuda.is_available() else "cpu"
pipe = DiffusionPipeline.from_pretrained(
    "common-canvas/CommonCanvas-S-NC",
    dtype=torch.float16,
    device_map=device # Changed "auto" to "cuda" as per error message
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/543 [00:00<?, ?B/s]

Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

scheduler_config.json:   0%|          | 0.00/346 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/460 [00:00<?, ?B/s]

...

vae/diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/911 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/553 [00:00<?, ?B/s]

Keyword arguments {'dtype': torch.float16} are not expected by StableDiffusionPipeline and will be ignored.


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

In [7]:
# Step 6: Generate images and audio

image_paths = []
audio_paths = []

for i, sent in enumerate(sentences):
    print(f"Processing sentence {i+1}/{len(sentences)}...")

    # Generate image
    img = pipe(sent).images[0]
    img_path = f"/content/images/image_{i}.png"
    img.save(img_path)
    image_paths.append(img_path)

    # Generate audio
    tts = gTTS(sent)
    aud_path = f"/content/audio/audio_{i}.mp3"
    tts.save(aud_path)
    audio_paths.append(aud_path)

Processing sentence 1/4...


  0%|          | 0/50 [00:00<?, ?it/s]

Processing sentence 2/4...


  0%|          | 0/50 [00:00<?, ?it/s]

Processing sentence 3/4...


  0%|          | 0/50 [00:00<?, ?it/s]

Processing sentence 4/4...


  0%|          | 0/50 [00:00<?, ?it/s]

In [8]:
# Step 7: Create video using OpenCV

frame_size = (640, 480)
fps = 1  # repeat frames according to audio length
video_name = "/content/final_news_video.avi"
fourcc = cv2.VideoWriter_fourcc(*"XVID")
out = cv2.VideoWriter(video_name, fourcc, fps, frame_size)

for img_path, aud_path in zip(image_paths, audio_paths):
    img = cv2.imread(img_path)
    img = cv2.resize(img, frame_size)

    audio = AudioSegment.from_mp3(aud_path)
    duration_sec = int(audio.duration_seconds)
    duration_sec = max(duration_sec, 1)

    for _ in range(duration_sec):
        out.write(img)

out.release()

In [9]:
# Step 8: Merge audio using pydub + ffmpeg

combined_audio = AudioSegment.empty()
for aud_path in audio_paths:
    combined_audio += AudioSegment.from_mp3(aud_path)
combined_audio.export("/content/final_audio.wav", format="wav")

# Merge video + audio
subprocess.call(
    'ffmpeg -y -i /content/final_news_video.avi -i /content/final_audio.wav -c:v copy -c:a aac /content/final_news_video.mp4',
    shell=True
)


0

In [10]:
# Step 9: Display video in Colab

mp4 = open("/content/final_news_video.mp4",'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f"""
<video width=640 controls>
      <source src="{data_url}" type="video/mp4">
</video>
""")